## Import statements

In [6]:
import numpy as np
import pandas as pd

# Data Prep

In [7]:
admissions = pd.read_csv('binary.csv')

# Make dummy variables for rank
data = pd.concat([admissions, pd.get_dummies(admissions['rank'], prefix='rank')], axis=1)
data = data.drop('rank', axis=1)

# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std
    
# Split off random 10% of the data for testing
np.random.seed(21)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.ix[sample], data.drop(sample)

# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

## Meat and potatoes?

In [12]:
# initialize random number generator
np.random.seed(21)


# activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# hyperparameters
n_hidden = 2 # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None

# initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                       size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** 0.5, 
                                        size=(n_hidden))

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward Pass ##
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = np.dot(weights_hidden_output, hidden_output)
        
        ## Backward Pass ##
        error = y - output
        
        output_error_term = error * output * (1 - output) #??? the reading makes it seem like this should be hidden layber output
        
        ## Propagate ##
        hidden_error = np.dot(weights_hidden_output, output_error_term) * \
        (hidden_output * (1 - hidden_output))
        
        hidden_error_term = 
        
        